<a href="https://colab.research.google.com/github/Pratyushanand26/WiDS_5.0/blob/main/Week2_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trigram Model

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
import random

In [ ]:
words=open('names.txt','r').read().splitlines()

In [ ]:
xs=[]
ys=[]

In [ ]:
chars = sorted(set(''.join(words))) + ['.']
stoi={}
for i in chars:
   for j in chars:
      stoi[i+j]=len(stoi)

In [ ]:
stoi2={}
stoi2['.']=0;
for c,i in enumerate(sorted(list(set(''.join(words))))):
  stoi2[i]=c+1

In [ ]:
itos2={}
itos2[0]='.';
for c,i in enumerate(sorted(list(set(''.join(words))))):
  itos2[c+1]=i

In [ ]:
itos={}
for k,v in stoi.items():
  itos[v]=k

In [ ]:
def dataset(words):
  xs = []
  ys = []

  for w in words:
    chs = ['.', '.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
      ix1 = stoi[ch1+ch2]
      ix2 = stoi2[ch3]
      xs.append(ix1)
      ys.append(ix2)
  X=torch.tensor(xs)
  Y=torch.tensor(ys)
  return X,Y

In [ ]:
random.shuffle(words)
trn=int(0.8*len(words))
dev=int(0.1*len(words))
test=int(0.1*len(words))

In [ ]:
Xtr,Ytr=dataset(words[:trn])
Xdev,Ydev=dataset(words[trn:trn+dev])
Xte,Yte=dataset(words[trn+dev:])

In [ ]:
# embd=F.one_hot(Xtr, num_classes=729).float()
# embd.size()

In [ ]:
# embd.size()

In [ ]:
W=torch.randn((729,27),requires_grad=True)

In [ ]:
Ytr.shape[0]

182447

In [ ]:
alpha=0.01
alpha*(W**2).mean()

tensor(0.0100, grad_fn=<MulBackward0>)

In [ ]:
W[1]

tensor([ 0.4281, -0.9889,  0.7290, -0.8484, -0.7939, -0.1268, -1.9154,  0.7054,
         0.8348,  0.9419,  0.3840, -0.4876, -0.2646,  0.4349,  0.9925,  0.6186,
        -1.6912, -1.1729, -0.3070,  0.3176,  0.3512,  0.0798,  0.1525,  0.4225,
         0.2132, -2.2108, -0.2847], grad_fn=<SelectBackward0>)

In [ ]:
W[1,2]

tensor(0.7290, grad_fn=<SelectBackward0>)

In [ ]:
#forward pass
for _ in range(50):
  logits = W[Xtr]
  # counts=logits.exp()
  # probs=counts/counts.sum(1,keepdims=True)
  # loss=-probs[torch.arange(Ytr.shape[0]),Ytr].log().mean()+ alpha*(W**2).mean()
  clf_loss=F.cross_entropy(logits,Ytr)
  reg_loss=alpha*(W**2).mean()
  loss=clf_loss+reg_loss
#backward-pass
  W.grad=None
  loss.backward()
#update
  W.data+=-50*W.grad

print(loss.item())

2.3773391246795654


In [ ]:
#dev-loss
embd=F.one_hot(Xdev, num_classes=729).float()
logits=embd@W
counts=logits.exp()
probs=counts/counts.sum(1,keepdims=True)
devloss=-probs[torch.arange(Ydev.shape[0]),Ydev].log().mean()+alpha*(W**2).mean()
print(devloss.item())

2.437612771987915


In [ ]:
Ytr.shape[0]

182447

In [ ]:
for _ in range(10):
  logs=[]
  context='..'
  while True:
    ix=stoi[context]
    embs=F.one_hot(torch.tensor([ix]),729).float()
    logits=embs@W
    count=logits.exp()
    probs=count/count.sum(1,keepdims=True)
    iy = torch.multinomial(probs, num_samples=1, replacement=True).item()
    logs.append(itos2[iy])
    context=context[1]+itos2[iy]
    if(iy==0):
      break
  print(''.join(logs))

jozouwayxvelin.
leighxgh.
nzfarahfhyoijxvvnmellpqbpvjbnzbknjvrgsbmrouiltnce.
lojgsvppeque.
hiipsqder.
tim.
abrfaristkekqbwhgcson.
sntxyq.
melomiliez.
norowxhcwegurbh.


In [ ]:
stoi['..']

728